# Week 8: Multimodality, RL, RAG - 8.2 LLM Agents

**Resource Requirements**: GPU with at least 8GB VRAM recommended for local model inference, or API access to language models

---

## Objectives

In this notebook, you will learn to:

1. **Understand the fundamentals of LLM agents** - Learn what agents are and how they work
2. **Build a simple arithmetic agent** - Create your first agent that can solve math problems
3. **Develop a complex navigation agent** - Build an agent that can navigate Wikipedia pages
4. **Master elicitation techniques** - Learn methods to improve agent performance

---

## 1. Introduction to LLM Agents

### What is an LLM Agent?

An LLM agent is a system where a language model interacts with external tools and environments to accomplish complex tasks. Think of it as giving an LLM the ability to:
- Use tools (like calculators, search engines, or APIs)
- Make decisions based on observations
- Take actions to achieve goals

### Core Components of an Agent

At its heart, an agent consists of:
1. **LLM**: The "brain" that makes decisions
2. **Tools**: External functions the LLM can call
3. **Environment**: The world the agent operates in
4. **Control Flow**: The loop that orchestrates interactions

The agent operates in a simple loop:
1. Observe the current state
2. Decide what action to take
3. Execute the action
4. Observe the result
5. Repeat until goal is achieved

In [ ]:
# Import necessary libraries
import os
import json
import re
from typing import Any, Dict, List, Optional, Callable
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
import openai
from openai import OpenAI
from rich import print as rprint
from rich.console import Console
from rich.panel import Panel

# Initialize console for better output formatting
console = Console()

# Set up OpenAI client (make sure to set your API key)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## 2. Building Our First Agent: Simple Arithmetic Calculator

Let's start by building a simple agent that can solve arithmetic problems. This will help us understand the basic patterns of agent development.

### Step 1: Define the Task Environment

In [ ]:
@dataclass
class ArithmeticProblem:
    """Represents an arithmetic problem for our agent to solve."""
    question: str
    answer: Optional[float] = None
    
    def check_answer(self, proposed_answer: float, tolerance: float = 0.001) -> bool:
        """Check if the proposed answer is correct within tolerance."""
        if self.answer is None:
            return False
        return abs(proposed_answer - self.answer) < tolerance
    
    def __str__(self):
        return f"Problem: {self.question}"

### Step 2: Create the Calculator Tool

Tools are the key to agent functionality. They allow the LLM to interact with the world. Each tool needs:
- A name
- A description (for the LLM to understand when to use it)
- An execution method

In [ ]:
class Tool(ABC):
    """Abstract base class for all tools."""
    
    @property
    @abstractmethod
    def name(self) -> str:
        """Return the name of the tool."""
        pass
    
    @property
    @abstractmethod
    def description(self) -> dict:
        """Return the OpenAI function description."""
        pass
    
    @abstractmethod
    def execute(self, **kwargs) -> str:
        """Execute the tool and return the result."""
        pass


class CalculatorTool(Tool):
    """A calculator tool that can evaluate mathematical expressions."""
    
    @property
    def name(self) -> str:
        return "calculator"
    
    @property
    def description(self) -> dict:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": "Evaluates a mathematical expression and returns the result. Supports basic arithmetic operations (+, -, *, /, **) and parentheses.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "The mathematical expression to evaluate (e.g., '2 + 2', '(5 * 3) - 7')"
                        }
                    },
                    "required": ["expression"],
                    "additionalProperties": False
                }
            }
        }
    
    def execute(self, expression: str, **kwargs) -> str:
        """Safely evaluate a mathematical expression."""
        try:
            # Remove any dangerous characters
            safe_expr = re.sub(r'[^0-9+\-*/()\s.]', '', expression)
            result = eval(safe_expr)
            return str(result)
        except (SyntaxError, NameError, ZeroDivisionError) as e:
            return f"Error: {type(e).__name__} - {str(e)}"
        except Exception as e:
            return f"Unexpected error: {str(e)}"

### Step 3: Build the Base Agent Class

Now let's create a base agent class that handles the core agent loop:

In [ ]:
@dataclass
class AgentConfig:
    """Configuration for an agent."""
    model: str = "gpt-4o-mini"
    temperature: float = 0.0
    max_steps: int = 10
    verbose: bool = True


class BaseAgent:
    """Base class for all agents."""
    
    def __init__(self, tools: List[Tool], config: AgentConfig = AgentConfig()):
        self.tools = {tool.name: tool for tool in tools}
        self.config = config
        self.history: List[Dict[str, Any]] = []
        self.client = OpenAI()
    
    def get_system_prompt(self) -> str:
        """Return the system prompt for the agent."""
        return """You are a helpful AI assistant with access to tools. Use these tools to help solve problems and answer questions. 
Think step by step about what you need to do, then use the appropriate tools to accomplish your goal."""
    
    def get_tool_descriptions(self) -> List[dict]:
        """Get descriptions of all available tools."""
        return [tool.description for tool in self.tools.values()]
    
    def execute_tool_call(self, tool_call) -> str:
        """Execute a tool call and return the result."""
        tool_name = tool_call.function.name
        if tool_name not in self.tools:
            return f"Error: Unknown tool '{tool_name}'"
        
        try:
            args = json.loads(tool_call.function.arguments)
            result = self.tools[tool_name].execute(**args)
            return result
        except Exception as e:
            return f"Error executing {tool_name}: {str(e)}"
    
    def get_llm_response(self, messages: List[Dict[str, Any]]) -> Any:
        """Get a response from the LLM."""
        response = self.client.chat.completions.create(
            model=self.config.model,
            messages=messages,
            tools=self.get_tool_descriptions(),
            temperature=self.config.temperature
        )
        return response.choices[0].message
    
    def log(self, message: str, style: str = "info"):
        """Log a message if verbose mode is enabled."""
        if self.config.verbose:
            styles = {
                "info": "[blue]",
                "success": "[green]",
                "error": "[red]",
                "tool": "[yellow]"
            }
            console.print(f"{styles.get(style, '')}[AGENT] {message}[/]")
    
    def run(self, task: str) -> str:
        """Run the agent on a given task."""
        # Initialize conversation
        messages = [
            {"role": "system", "content": self.get_system_prompt()},
            {"role": "user", "content": task}
        ]
        
        self.log(f"Starting task: {task}")
        
        for step in range(self.config.max_steps):
            self.log(f"Step {step + 1}/{self.config.max_steps}")
            
            # Get LLM response
            response = self.get_llm_response(messages)
            
            # If the LLM wants to use tools
            if response.tool_calls:
                messages.append(response.model_dump())
                
                for tool_call in response.tool_calls:
                    self.log(f"Calling tool: {tool_call.function.name}", "tool")
                    result = self.execute_tool_call(tool_call)
                    self.log(f"Tool result: {result}", "tool")
                    
                    # Add tool result to conversation
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": result
                    })
            else:
                # LLM provided a final answer
                self.log(f"Final answer: {response.content}", "success")
                return response.content
        
        self.log("Max steps reached without completion", "error")
        return "Failed to complete task within step limit."

### Step 4: Create the Arithmetic Agent

Now let's create a specialized agent for solving arithmetic problems:

In [ ]:
class ArithmeticAgent(BaseAgent):
    """An agent specialized in solving arithmetic problems."""
    
    def __init__(self, config: AgentConfig = AgentConfig()):
        # Initialize with calculator tool
        super().__init__(tools=[CalculatorTool()], config=config)
    
    def get_system_prompt(self) -> str:
        return """You are a mathematical problem solver. Your goal is to solve arithmetic problems step by step.
When given a problem:
1. Understand what the problem is asking
2. Break it down into steps if needed
3. Use the calculator tool to compute each step
4. Provide the final answer clearly

Always show your work and explain your reasoning."""
    
    def solve_problem(self, problem: ArithmeticProblem) -> Optional[float]:
        """Solve an arithmetic problem and return the numerical answer."""
        result = self.run(problem.question)
        
        # Extract numerical answer from the response
        numbers = re.findall(r'-?\d+\.?\d*', result)
        if numbers:
            try:
                return float(numbers[-1])  # Return the last number found
            except ValueError:
                return None
        return None

### Let's Test Our Arithmetic Agent!

In [ ]:
# Create some test problems
test_problems = [
    ArithmeticProblem(
        question="What is 25 times 4, plus 17?",
        answer=117
    ),
    ArithmeticProblem(
        question="If I have 150 apples and I eat 3 every day for 2 weeks, how many apples do I have left?",
        answer=108
    ),
    ArithmeticProblem(
        question="Calculate (12 + 8) * (15 - 7) / 4",
        answer=40
    )
]

# Test the agent
agent = ArithmeticAgent(config=AgentConfig(verbose=True))

for i, problem in enumerate(test_problems, 1):
    console.print(f"\n[bold]Problem {i}:[/bold]")
    console.print(Panel(str(problem), title="Problem", border_style="cyan"))
    
    answer = agent.solve_problem(problem)
    
    if answer is not None:
        is_correct = problem.check_answer(answer)
        status = " Correct" if is_correct else " Incorrect"
        color = "green" if is_correct else "red"
        console.print(f"\n[{color}]Agent's answer: {answer} ({status})[/{color}]")
        if problem.answer:
            console.print(f"Expected answer: {problem.answer}")
    else:
        console.print("\n[red]Agent failed to provide a numerical answer[/red]")

## 3. Building a Complex Agent: Wikipedia Navigator

Now let's build a more sophisticated agent that can play the Wikipedia game - navigating from one Wikipedia page to another using only links.

### Step 1: Define the Wikipedia Environment

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import unquote

@dataclass
class WikipediaGame:
    """Represents a Wikipedia navigation game."""
    start_page: str
    goal_page: str
    current_page: str = field(init=False)
    path: List[str] = field(default_factory=list)
    max_steps: int = 10
    
    def __post_init__(self):
        self.current_page = self.start_page
        self.path = [self.start_page]
    
    def is_complete(self) -> bool:
        """Check if the goal has been reached."""
        return self.current_page.lower() == self.goal_page.lower()
    
    def move_to_page(self, page_title: str) -> bool:
        """Move to a new page if it's valid."""
        # In a real implementation, we'd validate this is a real link
        self.current_page = page_title
        self.path.append(page_title)
        return True
    
    def get_status(self) -> str:
        """Get the current game status."""
        return f"Current page: {self.current_page}\nGoal: {self.goal_page}\nSteps taken: {len(self.path) - 1}/{self.max_steps}"

### Step 2: Create Wikipedia Tools

In [ ]:
class WikipediaContentTool(Tool):
    """Tool to get Wikipedia page content with links."""
    
    @property
    def name(self) -> str:
        return "get_wikipedia_content"
    
    @property
    def description(self) -> dict:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": "Get the content of a Wikipedia page including available links to other pages. Links are shown in [brackets].",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "page_title": {
                            "type": "string",
                            "description": "The title of the Wikipedia page to retrieve"
                        }
                    },
                    "required": ["page_title"],
                    "additionalProperties": False
                }
            }
        }
    
    def execute(self, page_title: str, **kwargs) -> str:
        """Get Wikipedia page content (simplified version)."""
        try:
            # For demo purposes, return mock content
            # In a real implementation, this would fetch actual Wikipedia content
            mock_content = {
                "Albert Einstein": """Albert Einstein was a theoretical physicist who developed the theory of relativity.
He is best known for his equation E=mc². Born in [Germany], he later moved to the [United States].
His work influenced [quantum mechanics] and [cosmology]. He won the [Nobel Prize] in Physics in 1921.""",
                
                "Germany": """Germany is a country in Central Europe. It has a rich history including the [Holy Roman Empire],
[World War I], and [World War II]. Famous Germans include [Albert Einstein], [Ludwig van Beethoven], and [Angela Merkel].
The capital is [Berlin].""",
                
                "Nobel Prize": """The Nobel Prize is a set of annual international awards. Categories include [Physics],
[Chemistry], [Medicine], [Literature], and [Peace]. Notable winners include [Albert Einstein], [Marie Curie],
and [Martin Luther King Jr.]. The prizes were established by [Alfred Nobel].""",
                
                "Physics": """Physics is the natural science that studies matter, energy, and their interactions.
Major branches include [classical mechanics], [quantum mechanics], [thermodynamics], and [relativity].
Famous physicists include [Isaac Newton], [Albert Einstein], and [Stephen Hawking]."""
            }
            
            content = mock_content.get(page_title, f"Page '{page_title}' contains various information and links to other topics.")
            return f"Content of '{page_title}':\n\n{content}"
            
        except Exception as e:
            return f"Error retrieving page: {str(e)}"


class WikipediaNavigateTool(Tool):
    """Tool to navigate to a new Wikipedia page."""
    
    def __init__(self, game: WikipediaGame):
        self.game = game
    
    @property
    def name(self) -> str:
        return "navigate_to_page"
    
    @property
    def description(self) -> dict:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": "Navigate to a Wikipedia page that is linked from the current page.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "page_title": {
                            "type": "string",
                            "description": "The title of the Wikipedia page to navigate to (must be a link from current page)"
                        }
                    },
                    "required": ["page_title"],
                    "additionalProperties": False
                }
            }
        }
    
    def execute(self, page_title: str, **kwargs) -> str:
        """Navigate to a new page."""
        if self.game.is_complete():
            return "Game is already complete!"
        
        if len(self.game.path) >= self.game.max_steps:
            return f"Maximum steps ({self.game.max_steps}) reached! Game over."
        
        success = self.game.move_to_page(page_title)
        if success:
            status = f"Successfully navigated to '{page_title}'.\n{self.game.get_status()}"
            if self.game.is_complete():
                status += f"\n\n<‰ Congratulations! You've reached the goal in {len(self.game.path) - 1} steps!"
            return status
        else:
            return f"Failed to navigate to '{page_title}'. Make sure it's a valid link from the current page."

### Step 3: Create the Wikipedia Navigator Agent

In [ ]:
class WikipediaNavigatorAgent(BaseAgent):
    """An agent that can navigate Wikipedia to reach a goal page."""
    
    def __init__(self, game: WikipediaGame, config: AgentConfig = AgentConfig()):
        self.game = game
        tools = [
            WikipediaContentTool(),
            WikipediaNavigateTool(game)
        ]
        super().__init__(tools=tools, config=config)
    
    def get_system_prompt(self) -> str:
        return f"""You are playing the Wikipedia game. Your goal is to navigate from '{self.game.start_page}' to '{self.game.goal_page}' using only links found on Wikipedia pages.

Strategy tips:
1. First, get the content of your current page to see available links
2. Look for links that might be related to your goal
3. Think about the conceptual path from your current topic to the goal
4. Navigate strategically - sometimes you need to go through intermediate topics
5. Remember you have a maximum of {self.game.max_steps} steps

Current game status:
{self.game.get_status()}"""
    
    def play(self) -> bool:
        """Play the Wikipedia game and return True if successful."""
        task = f"""Navigate from '{self.game.start_page}' to '{self.game.goal_page}'.
Start by getting the content of the current page, then navigate through links to reach your goal."""
        
        result = self.run(task)
        return self.game.is_complete()

### Let's Test the Wikipedia Navigator!

In [ ]:
# Create a Wikipedia game
game = WikipediaGame(
    start_page="Albert Einstein",
    goal_page="Physics",
    max_steps=5
)

# Create and run the agent
wiki_agent = WikipediaNavigatorAgent(
    game=game,
    config=AgentConfig(verbose=True, max_steps=10)
)

console.print(Panel(
    f"Starting Wikipedia Game\nFrom: {game.start_page}\nTo: {game.goal_page}",
    title="Game Setup",
    border_style="green"
))

# Play the game
success = wiki_agent.play()

# Show results
if success:
    console.print("\n[green] Successfully completed the game![/green]")
    console.print(f"Path taken: {' ’ '.join(game.path)}")
else:
    console.print("\n[red] Failed to reach the goal[/red]")
    console.print(f"Path taken: {' ’ '.join(game.path)}")

## 4. Advanced Techniques: Elicitation

Elicitation refers to techniques for improving agent performance. Let's explore several approaches:

### 4.1 Enhanced Prompting

One of the simplest ways to improve agent performance is through better prompting:

In [ ]:
class EnhancedWikipediaAgent(WikipediaNavigatorAgent):
    """Wikipedia agent with enhanced prompting strategies."""
    
    def get_system_prompt(self) -> str:
        # Get path history for context
        path_str = " ’ ".join(self.game.path) if self.game.path else "No moves yet"
        
        return f"""You are an expert Wikipedia navigator. Your goal is to find the shortest path from '{self.game.start_page}' to '{self.game.goal_page}'.

CURRENT STATUS:
- Current page: {self.game.current_page}
- Goal page: {self.game.goal_page}
- Steps taken: {len(self.game.path) - 1}/{self.game.max_steps}
- Path so far: {path_str}

STRATEGY GUIDELINES:
1. Analyze semantic relationships between current page and goal
2. Identify conceptual bridges (e.g., person ’ field they work in ’ specific topic)
3. Prefer direct links when available
4. Avoid backtracking unless necessary
5. Consider these connection patterns:
   - Person ’ Their field ’ Concepts in that field
   - Specific ’ General ’ Different specific
   - Historical events ’ Time periods ’ Other events
   - Geographic locations ’ Countries ’ Related topics

THINKING PROCESS:
Before each move, consider:
- What is the conceptual distance to the goal?
- Which available links move me closer?
- Am I avoiding circular paths?"""

### 4.2 ReAct Pattern: Reasoning + Acting

The ReAct pattern separates reasoning from action, making the agent's thought process more explicit:

In [ ]:
class ReActTool(Tool):
    """A tool that forces the agent to reason before acting."""
    
    @property
    def name(self) -> str:
        return "think"
    
    @property
    def description(self) -> dict:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": "Record your reasoning and planning before taking action. Use this to think through your strategy.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "reasoning": {
                            "type": "string",
                            "description": "Your analysis of the current situation and planned next steps"
                        }
                    },
                    "required": ["reasoning"],
                    "additionalProperties": False
                }
            }
        }
    
    def execute(self, reasoning: str, **kwargs) -> str:
        """Simply acknowledge the reasoning."""
        return "Reasoning recorded. Now proceed with your action based on this analysis."


class ReActWikipediaAgent(WikipediaNavigatorAgent):
    """Wikipedia agent using ReAct pattern."""
    
    def __init__(self, game: WikipediaGame, config: AgentConfig = AgentConfig()):
        super().__init__(game, config)
        # Add the think tool
        self.tools["think"] = ReActTool()
    
    def get_system_prompt(self) -> str:
        base_prompt = super().get_system_prompt()
        return base_prompt + """\n\nIMPORTANT: Use the ReAct pattern:
1. THINK: Before each action, use the 'think' tool to analyze the situation
2. ACT: Then take the appropriate action based on your reasoning
3. OBSERVE: After acting, observe the results before thinking again

This helps ensure thoughtful, strategic navigation rather than random clicking."""

### 4.3 Path Planning Tool

Advanced agents can benefit from tools that help them plan and test strategies:

In [ ]:
class PathPlannerTool(Tool):
    """A tool that helps plan potential paths without committing to them."""
    
    def __init__(self, content_tool: WikipediaContentTool):
        self.content_tool = content_tool
    
    @property
    def name(self) -> str:
        return "plan_path"
    
    @property
    def description(self) -> dict:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": "Explore a potential path by checking what links are available on a page without actually navigating there.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "page_title": {
                            "type": "string",
                            "description": "The page to explore (without navigating to it)"
                        },
                        "target": {
                            "type": "string",
                            "description": "The target page you're trying to reach"
                        }
                    },
                    "required": ["page_title", "target"],
                    "additionalProperties": False
                }
            }
        }
    
    def execute(self, page_title: str, target: str, **kwargs) -> str:
        """Check if a page might lead to the target."""
        content = self.content_tool.execute(page_title)
        
        # Check if target is directly linked
        if f"[{target}]" in content:
            return f" Great choice! '{page_title}' has a direct link to '{target}'."
        
        # Look for related concepts
        links = re.findall(r'\[([^\]]+)\]', content)
        related = [link for link in links if any(word in link.lower() for word in target.lower().split())]
        
        if related:
            return f"Promising: '{page_title}' has related links: {', '.join(related[:3])}"
        else:
            return f"Not ideal: '{page_title}' doesn't seem to have links closely related to '{target}'."

### 4.4 Memory-Enhanced Agent

Let's create an agent that maintains better memory of visited pages:

In [ ]:
@dataclass
class PageMemory:
    """Store information about visited pages."""
    title: str
    links: List[str]
    summary: str
    visited_step: int


class MemoryEnhancedAgent(WikipediaNavigatorAgent):
    """Agent with enhanced memory of visited pages."""
    
    def __init__(self, game: WikipediaGame, config: AgentConfig = AgentConfig()):
        super().__init__(game, config)
        self.page_memory: Dict[str, PageMemory] = {}
        self.current_step = 0
    
    def execute_tool_call(self, tool_call) -> str:
        """Override to capture page information in memory."""
        result = super().execute_tool_call(tool_call)
        
        # If this was a content retrieval, store in memory
        if tool_call.function.name == "get_wikipedia_content":
            args = json.loads(tool_call.function.arguments)
            page_title = args.get("page_title", "")
            
            # Extract links from content
            links = re.findall(r'\[([^\]]+)\]', result)
            
            # Create summary (first 100 chars)
            summary = result.split('\n\n')[1][:100] if '\n\n' in result else result[:100]
            
            self.page_memory[page_title] = PageMemory(
                title=page_title,
                links=links,
                summary=summary,
                visited_step=self.current_step
            )
            self.current_step += 1
        
        return result
    
    def get_system_prompt(self) -> str:
        base_prompt = super().get_system_prompt()
        
        # Add memory context
        if self.page_memory:
            memory_str = "\n\nPAGE MEMORY:\n"
            for page, mem in self.page_memory.items():
                memory_str += f"- {page}: {mem.summary}... (Links: {', '.join(mem.links[:3])}...)\n"
            
            base_prompt += memory_str
        
        return base_prompt

### Testing Advanced Agents

Let's compare the performance of different agent strategies:

In [ ]:
def test_agent_strategy(agent_class, game_config: dict, agent_name: str):
    """Test an agent strategy and return performance metrics."""
    game = WikipediaGame(**game_config)
    agent = agent_class(game=game, config=AgentConfig(verbose=False, max_steps=15))
    
    console.print(f"\n[bold]Testing {agent_name}[/bold]")
    success = agent.play()
    
    steps_taken = len(game.path) - 1 if game.path else 0
    
    # Print results
    if success:
        console.print(f"[green] Success in {steps_taken} steps[/green]")
        console.print(f"Path: {' ’ '.join(game.path)}")
    else:
        console.print(f"[red] Failed after {steps_taken} steps[/red]")
        console.print(f"Path: {' ’ '.join(game.path)}")
    
    return {"success": success, "steps": steps_taken, "path": game.path}


# Test different strategies on the same problem
test_game = {
    "start_page": "Albert Einstein",
    "goal_page": "Nobel Prize",
    "max_steps": 6
}

strategies = [
    (WikipediaNavigatorAgent, "Basic Agent"),
    (EnhancedWikipediaAgent, "Enhanced Prompting"),
    (ReActWikipediaAgent, "ReAct Pattern"),
    (MemoryEnhancedAgent, "Memory Enhanced")
]

results = []
for agent_class, name in strategies:
    result = test_agent_strategy(agent_class, test_game, name)
    results.append((name, result))

# Summary
console.print("\n[bold]Performance Summary:[/bold]")
for name, result in results:
    status = "" if result["success"] else ""
    console.print(f"{status} {name}: {result['steps']} steps")

## 5. Key Takeaways and Best Practices

### Agent Design Principles

1. **Start Simple**: Begin with basic tools and gradually add complexity
2. **Clear Tool Descriptions**: LLMs rely on tool descriptions to understand when and how to use them
3. **Robust Error Handling**: Agents should gracefully handle tool failures
4. **Iterative Improvement**: Use elicitation techniques to enhance performance

### Common Patterns

1. **Tool Calling Loop**: The basic pattern of observe ’ decide ’ act ’ repeat
2. **Memory Management**: Maintaining context across steps
3. **Strategy Injection**: Using prompts to guide agent behavior
4. **Reflection**: Having agents reason about their actions

### Performance Optimization

1. **Prompt Engineering**: Better prompts lead to better agent behavior
2. **Tool Design**: Well-designed tools make tasks easier
3. **Context Management**: Balance between providing enough context and avoiding confusion
4. **Elicitation**: Different techniques work better for different tasks

### Future Directions

Agent research is rapidly evolving. Key areas include:
- Multi-agent collaboration
- Long-term memory and learning
- More sophisticated planning algorithms
- Integration with other AI systems (vision, robotics, etc.)

## Exercises

1. **Extend the Arithmetic Agent**: Add support for more complex operations (square roots, trigonometry, etc.)

2. **Create a New Tool**: Design and implement a tool that helps the Wikipedia agent by providing category information

3. **Implement a New Elicitation Strategy**: Try implementing a "chain of thought" approach where the agent explicitly breaks down its reasoning

4. **Build Your Own Agent**: Create an agent for a different domain (e.g., code debugging, recipe following, game playing)

5. **Comparative Analysis**: Run experiments comparing different elicitation strategies on various Wikipedia navigation tasks

## Conclusion

In this notebook, we've explored the fundamentals of LLM agents:
- Built simple and complex agents from scratch
- Learned how tool calling enables LLMs to interact with the world
- Discovered various elicitation techniques to improve performance
- Understood the key patterns and principles of agent design

Agents represent a powerful paradigm for extending LLM capabilities beyond text generation, enabling them to solve complex, multi-step problems in interactive environments. As you continue exploring this field, remember that the key to effective agents lies in thoughtful design of tools, prompts, and control flows.